## 1. External Stage

### - Створіть External Stage, який підключено до публічного бакету

In [ ]:
CREATE STORAGE INTEGRATION ovorobiov_s3_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::554739427960:role/ovorobiov-snowflake-role'
  STORAGE_ALLOWED_LOCATIONS = ('*');

In [ ]:
DESC INTEGRATION ovorobiov_s3_int;

In [ ]:
CREATE OR REPLACE STAGE ovorobiov_s3_stagre
STORAGE_INTEGRATION = ovorobiov_s3_int
URL = 's3://robot-dreams-source-data/'
FILE_FORMAT = ovorobiov_parquet_format;

### - Перевірте доступність файлів за допомогою LIST @stage_name

In [ ]:
LIST @ovorobiov_s3_stagre

## 2. Завантаження сирих даних
* Створіть таблиці yellow_raw, green_raw, taxi_zone_lookup
* Завантажте файли в таблиці за допомогою команди COPY INTO

In [ ]:
CREATE OR REPLACE FILE FORMAT ovorobiov_csv_format
    TYPE = 'CSV'
    COMPRESSION = 'AUTO'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    TRIM_SPACE = TRUE
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
;

In [ ]:
CREATE OR REPLACE FILE FORMAT ovorobiov_parquet_format
    TYPE = 'PARQUET'
    COMPRESSION = 'AUTO'
    USE_LOGICAL_TYPE = TRUE
    TRIM_SPACE = TRUE
;

In [ ]:
CREATE OR REPLACE TABLE TAXI_ZONE_LOOKUP (
    LOCATION VARCHAR,
    BOROUGH VARCHAR,
    ZONE VARCHAR,
    SERVICE_ZONE VARCHAR
);

In [ ]:
CREATE OR REPLACE TABLE yellow_taxi_raw (
    VendorID               BIGINT,
    pickup_datetime        TIMESTAMP,
    dropoff_datetime       TIMESTAMP,
    passenger_count        BIGINT,
    trip_distance          DOUBLE,
    RatecodeID             BIGINT,
    store_and_fwd_flag     STRING,
    PULocationID           BIGINT,
    DOLocationID           BIGINT,
    payment_type           BIGINT,
    fare_amount            DOUBLE,
    extra                  DOUBLE,
    mta_tax                DOUBLE,
    tip_amount             DOUBLE,
    tolls_amount           DOUBLE,
    improvement_surcharge  DOUBLE,
    total_amount           DOUBLE,
    congestion_surcharge   DOUBLE,
    airport_fee            DOUBLE
);

In [ ]:
CREATE OR REPLACE TABLE green_taxi_raw (
    VendorID BIGINT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count BIGINT,
    trip_distance DOUBLE,
    RatecodeID BIGINT,
    store_and_fwd_flag STRING,
    PULocationID BIGINT,
    DOLocationID BIGINT,
    payment_type BIGINT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    total_amount DOUBLE,
    congestion_surcharge DOUBLE,
    ehail_fee DOUBLE,
    trip_type DOUBLE
);

In [ ]:
COPY INTO TAXI_ZONE_LOOKUP
FROM @ovorobiov_s3_stagre/home-work-1/nyc_taxi/taxi_zone_lookup.csv
FILE_FORMAT = (FORMAT_NAME = ovorobiov_csv_format)

In [ ]:
COPY INTO yellow_taxi_raw
FROM (
    SELECT 
        $1:VendorID::BIGINT,
        COALESCE($1:tpep_pickup_datetime::TIMESTAMP, $1:lpep_pickup_datetime::TIMESTAMP) AS pickup_datetime,
        COALESCE($1:tpep_dropoff_datetime::TIMESTAMP, $1:lpep_dropoff_datetime::TIMESTAMP) AS dropoff_datetime,
        $1:passenger_count::BIGINT,
        $1:trip_distance::DOUBLE,
        $1:RatecodeID::BIGINT,
        $1:store_and_fwd_flag::STRING,
        $1:PULocationID::BIGINT,
        $1:DOLocationID::BIGINT,
        $1:payment_type::BIGINT,
        $1:fare_amount::DOUBLE,
        $1:extra::DOUBLE,
        $1:mta_tax::DOUBLE,
        $1:tip_amount::DOUBLE,
        $1:tolls_amount::DOUBLE,
        $1:improvement_surcharge::DOUBLE,
        $1:total_amount::DOUBLE,
        $1:congestion_surcharge::DOUBLE,
        COALESCE($1:airport_fee, $1:Airport_fee)::DOUBLE AS airport_fee
    FROM @ovorobiov_s3_stagre/home-work-1/nyc_taxi/yellow/
);

In [ ]:
COPY INTO green_taxi_raw
FROM (
    SELECT
        $1:VendorID::BIGINT AS VendorID,
        COALESCE($1:lpep_pickup_datetime, $1:pickup_datetime)::TIMESTAMP AS pickup_datetime,
        COALESCE($1:lpep_dropoff_datetime, $1:dropoff_datetime)::TIMESTAMP AS dropoff_datetime,
        $1:passenger_count::BIGINT AS passenger_count,
        $1:trip_distance::DOUBLE AS trip_distance,
        $1:RatecodeID::BIGINT AS RatecodeID,
        $1:store_and_fwd_flag::STRING AS store_and_fwd_flag,
        $1:PULocationID::BIGINT AS PULocationID,
        $1:DOLocationID::BIGINT AS DOLocationID,
        $1:payment_type::BIGINT AS payment_type,
        $1:fare_amount::DOUBLE AS fare_amount,
        $1:extra::DOUBLE AS extra,
        $1:mta_tax::DOUBLE AS mta_tax,
        $1:tip_amount::DOUBLE AS tip_amount,
        $1:tolls_amount::DOUBLE AS tolls_amount,
        $1:improvement_surcharge::DOUBLE AS improvement_surcharge,
        $1:total_amount::DOUBLE AS total_amount,
        $1:congestion_surcharge::DOUBLE AS congestion_surcharge,
        $1:ehail_fee::DOUBLE AS ehail_fee,
        $1:trip_type::DOUBLE AS trip_type
    FROM @ovorobiov_s3_stagre/home-work-1/nyc_taxi/green/
);

## 3. Збагачення даних
* Створіть таблиці yellow_enriched та green_enriched
* Реалізуйте JOIN з lookup таблицею
* Pickup/Dropoff зони (zone_name)
* Використовуйте LEFT JOIN для збереження всіх записів

In [ ]:
CREATE OR REPLACE TABLE yellow_enriched AS
SELECT 
    yt.VendorID,
    yt.pickup_datetime,
    yt.dropoff_datetime,
    yt.passenger_count,
    yt.trip_distance,
    yt.RatecodeID,
    yt.store_and_fwd_flag,
    yt.PULocationID,
    yt.DOLocationID,
    yt.payment_type,
    yt.fare_amount,
    yt.extra,
    yt.mta_tax,
    yt.tip_amount,
    yt.tolls_amount,
    yt.improvement_surcharge,
    yt.total_amount,
    yt.congestion_surcharge,
    yt.airport_fee,
    ptz.zone AS pickup_zone,
    dtz.zone AS dropoff_zone
FROM YELLOW_TAXI_RAW yt
LEFT JOIN TAXI_ZONE_LOOKUP ptz ON yt.PULOCATIONID = ptz.LOCATION
LEFT JOIN TAXI_ZONE_LOOKUP dtz ON yt.DOLOCATIONID = dtz.LOCATION ;

In [ ]:
CREATE OR REPLACE TABLE green_enriched AS
SELECT
    gt.VendorID,
    gt.pickup_datetime,
    gt.dropoff_datetime,
    gt.passenger_count,
    gt.trip_distance,
    gt.RatecodeID,
    gt.store_and_fwd_flag,
    gt.PULocationID,
    gt.DOLocationID,
    gt.payment_type,
    gt.fare_amount,
    gt.extra,
    gt.mta_tax,
    gt.tip_amount,
    gt.tolls_amount,
    gt.improvement_surcharge,
    gt.total_amount,
    gt.congestion_surcharge,
    gt.ehail_fee,
    gt.trip_type,
    ptz.zone AS pickup_zone,
    dtz.zone AS dropoff_zone
FROM GREEN_TAXI_RAW gt
LEFT JOIN TAXI_ZONE_LOOKUP ptz ON gt.PULOCATIONID = ptz.LOCATION
LEFT JOIN TAXI_ZONE_LOOKUP dtz ON gt.DOLOCATIONID = dtz.LOCATION;

## 4. Трансформації
* Фільтруйте записи:
  * trip_distance > 0
  * total_amount > 0
  * passenger_count між 1 та 6
* Додайте колонку trip_category:
  * Short (до 2 км), Medium (2–10 км), Long (>10 км)
* Додайте колонку pickup_hour: годину з pickup_datetime
* Створіть агреговану таблицю з підрахунком по зонах

In [ ]:
DELETE FROM GREEN_ENRICHED WHERE trip_distance > 0 AND total_amount > 0 AND (passenger_count >= 1 AND passenger_count <= 6);

In [ ]:
DELETE FROM YELLOW_ENRICHED WHERE trip_distance > 0 AND total_amount > 0 AND (passenger_count >= 1 AND passenger_count <= 6);

In [ ]:
ALTER TABLE GREEN_ENRICHED ADD COLUMN pickup_hour INT;
ALTER TABLE GREEN_ENRICHED ADD COLUMN trip_category VARCHAR;
ALTER TABLE YELLOW_ENRICHED ADD COLUMN pickup_hour INT;
ALTER TABLE YELLOW_ENRICHED ADD COLUMN trip_category VARCHAR;


In [ ]:
-- Додайте колонку trip_category:
--- Short (до 2 км), Medium (2–10 км), Long (>10 км)
--- Додайте колонку pickup_hour: годину з pickup_datetime
UPDATE GREEN_ENRICHED SET 
    trip_category = CASE 
        WHEN trip_distance < 2 THEN 'Short'
        WHEN (trip_distance >= 2 AND trip_distance <= 10) THEN 'Medium'
        ELSE 'Long'
        END,
    pickup_hour = EXTRACT(HOUR FROM pickup_datetime);

-- Додайте колонку trip_category:
--- Short (до 2 км), Medium (2–10 км), Long (>10 км)
--- Додайте колонку pickup_hour: годину з pickup_datetime
UPDATE YELLOW_ENRICHED SET 
    trip_category = CASE 
        WHEN trip_distance < 2 THEN 'Short'
        WHEN (trip_distance >= 2 AND trip_distance <= 10) THEN 'Medium'
        ELSE 'Long'
        END,
    pickup_hour = EXTRACT(HOUR FROM pickup_datetime);



In [ ]:
CREATE OR REPLACE TABLE GREEN_ZONE_SUMMARY AS
SELECT 
    pickup_zone,
    count(*) AS total_trips,
    avg(trip_distance) AS avg_trip_distance,
    avg(total_amount) AS avg_total_amount,
    avg(tip_amount) AS avg_tip_amount,
    max(trip_distance) AS max_trip_distance,
    min(tip_amount) AS min_tip_amount
FROM GREEN_ENRICHED
GROUP BY pickup_zone;

In [ ]:
CREATE OR REPLACE TABLE YELLOW_ZONE_SUMMARY AS
SELECT 
    pickup_zone,
    count(*) AS total_trips,
    avg(trip_distance) AS avg_trip_distance,
    avg(total_amount) AS avg_total_amount,
    avg(tip_amount) AS avg_tip_amount,
    max(trip_distance) AS max_trip_distance,
    min(tip_amount) AS min_tip_amount
FROM YELLOW_ENRICHED
GROUP BY pickup_zone;

## 5. Time Travel

* Видаліть кілька записів зі збагаченої таблиці (наприклад, green_enriched)
* За допомогою Time Travel:
  * Перевірте стару версію таблиці (через AT або BEFORE)
  * Відновіть видалені записи у нову таблицю або у ту ж (через INSERT SELECT)

In [ ]:
SELECT count(*) FROM GREEN_ENRICHED WHERE TOTAL_AMOUNT = 1.1;

In [ ]:
-- 1. Delete a few records from green_enriched
DELETE FROM green_enriched WHERE TOTAL_AMOUNT = 1.1;

In [ ]:
SELECT query_id, query_text, start_time
FROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY(
    END_TIME_RANGE_START => DATEADD('hour', -1, CURRENT_TIMESTAMP),
    END_TIME_RANGE_END   => CURRENT_TIMESTAMP
))
WHERE query_text ILIKE 'DELETE%'
ORDER BY start_time DESC;

In [ ]:
-- 2. View an old version of the table using Time Travel
SELECT count(*) FROM green_enriched AT (STATEMENT => '01be64eb-0001-660f-0001-600a0012f71e');

In [ ]:
-- 3. Restore deleted data into the new table
CREATE OR REPLACE TABLE deleted_green_log AS
SELECT *
FROM green_enriched
AT (OFFSET => -60*5);

In [ ]:
-- 4. Check the new table
SELECT TOP 10 * FROM deleted_green_log;

## 6. Streams

* Створіть Stream на таблиці yellow_enriched
* Додайте нові записи вручну або через COPY INTO
* Перевірте, що Stream відображає INSERT/UPDATE зміни
* Створіть цільову таблицю yellow_changes_log для зберігання змін

In [ ]:
-- 1. Create stream
CREATE OR REPLACE STREAM yellow_enriched_stream
ON TABLE yellow_enriched;


In [ ]:
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'YELLOW_ENRICHED';

In [ ]:
SELECT TOP 1 * FROM yellow_enriched;

In [ ]:
-- 2. Insert new data manually
INSERT INTO yellow_enriched (
    PICKUP_DATETIME,
    DROPOFF_DATETIME,
    DOLOCATIONID,
    VENDORID,
    PASSENGER_COUNT,
    PICKUP_ZONE,
    PULOCATIONID,
    TOTAL_AMOUNT
)
VALUES
('2017-07-05 12:58:50', '2017-07-05 12:58:54', 264, 1, 2, 'Springfield Gardens South', 219, 333),
('2017-07-08 14:58:50', '2017-07-08 18:58:54', 264, 1, 2, 'Springfield Gardens South', 219, 444);

In [ ]:
-- 3. View stream contents (shows changes)
SELECT * FROM yellow_enriched_stream;


In [ ]:
-- 4. Create target table for changes log
CREATE OR REPLACE TABLE yellow_changes_log AS
SELECT * FROM yellow_enriched_stream WHERE 1=0;

In [ ]:
-- Alter METADATA$ROW_ID AND METADATA$ACTION columns to support longer IDs
ALTER TABLE yellow_changes_log 
    ALTER COLUMN "METADATA$ROW_ID" SET DATA TYPE VARCHAR;
ALTER TABLE yellow_changes_log 
    ALTER COLUMN "METADATA$ACTION" SET DATA TYPE VARCHAR;

In [ ]:
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'YELLOW_CHANGES_LOG'; 

## 7. Tasks (batch/streaming jobs)

* Створіть Task, який щогодини:
  * Зчитує зміни зі Stream
  * Вставляє нові записи у yellow_changes_log
* Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску
* Зробіть окремий Task для агрегованої статистики:
  * Середня відстань
  * Середня ціна
  * Кількість поїздок
  * Зберігайте у таблицю zone_hourly_stats
* Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску



In [ ]:
-- Create task to read and save changes from stream
CREATE OR REPLACE TASK task_yellow_changes_log
  WAREHOUSE = SNOWFLAKE_LEARNING_WH
  SCHEDULE = '1 HOUR'
AS
INSERT INTO yellow_changes_log
SELECT *
FROM yellow_enriched_stream;

In [ ]:
-- Create table for aggregated
CREATE OR REPLACE TABLE zone_hourly_stats (
    stat_hour INT,
    avg_distance FLOAT,
    avg_fare FLOAT,
    trip_count INT
);

In [ ]:
-- Create task for aggregated stats
CREATE OR REPLACE TASK task_zone_hourly_stats
  WAREHOUSE = SNOWFLAKE_LEARNING_WH
  SCHEDULE = '1 HOUR'
AS
INSERT INTO zone_hourly_stats
SELECT 
    pickup_hour AS stat_hour,
    AVG(trip_distance) AS avg_distance,
    AVG(fare_amount) AS avg_fare,
    COUNT(*) AS trip_count
FROM yellow_enriched
GROUP BY pickup_hour
ORDER BY 1;

## Start tasks: task_zone_hourly_stats and task_yellow_changes_log

In [ ]:
ALTER TASK task_zone_hourly_stats RESUME;

In [ ]:
ALTER TASK task_yellow_changes_log RESUME;

## Execute tasks immediately: task_zone_hourly_stats and task_yellow_changes_log

In [ ]:
EXECUTE TASK task_zone_hourly_stats;


In [ ]:
EXECUTE TASK task_yellow_changes_log;

## Check results of tasks execution

In [ ]:
SELECT * FROM zone_hourly_stats;

In [ ]:
SELECT * FROM yellow_changes_log;